# Librairies

In [ ]:
# ACTIVER GPU + RAM ELEVEE

In [ ]:
#!pip install keras_gpt_2
#from keras_gpt_2 import load_trained_model_from_checkpoint, get_bpe_from_files, generate

import os
from datetime import datetime
from time import strftime
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd

!pip install -q gpt-2-simple
import gpt_2_simple as gpt2

gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

# Répertoires / paramètres

In [ ]:
data_rep = 'drive/MyDrive/ML/Datasets/'
output_rep = 'drive/My Drive/Generated_lyrics/' # pour stockage des lyrics générées

model_folder = 'drive/My Drive/ML/gpt2/models'
checkpoint_folder = 'drive/My Drive/ML/gpt2/checkpoint'

MODEL_in = '124M'
MODEL_out = '124M_AllRapFR' # nom du modèle en entrée

# Préparation

**Téléchargement GPT2 small**

In [ ]:
gpt2.download_gpt2(model_name = MODEL_in)

Fetching checkpoint: 1.05Mit [00:00, 540Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 2.37Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 760Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:18, 26.3Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 354Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 3.69Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 3.10Mit/s]


**Check bon import**

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, 
               model_name = MODEL_in)

gpt2.generate(sess,
              model_name = MODEL_in,
              length = 100,
              temperature = 0.7,
              top_p = 0.95,
              top_k = 40,
              prefix = 'What about generating rap lyrics?',
              nsamples = 3,
              batch_size = 3)

Loading pretrained model models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
What about generating rap lyrics? That's where the real sense comes in. A lot of the R&B producers that I've worked with have been musicians that were very vocal in their rap lyrics. They're not that creative. They're not that bold. And they're not that big on rap.

I see a lot of rappers doing their own thing on the album. Do you ever see that being a big part of the album?

I think it's a big part of the album. I think it
What about generating rap lyrics?

We have a lot of people who are working for the company and we're working on a lot of things, but you can't really focus on the songs. So there's a lot of specific things that we want to do. There are a lot of different things that we want to do for the music.

We're also going to try to focus on that kind of thing, which is a lot of the time we have to do it because it's just too much.

What about generating rap lyrics?

**Input : conversion en tokens / archive avant de lancer l'entrainement (OOM sinon)**

In [ ]:
# =============================================================================
# txt = couplets concatenés avec balises start/endoftext pour chaque couplet (voir 00-Data_clean.ipynb)
# =============================================================================

lyrics_txt = os.path.join(data_rep, 'all_fr_verse.txt')
lyrics_npz = os.path.join(data_rep, 'all_fr_verse.npz')

gpt2.encode_dataset(file_path = lyrics_txt,
                    model_name = MODEL_in,
                    out_path = lyrics_npz)

Reading files


100%|██████████| 1/1 [06:09<00:00, 369.55s/it]


Writing drive/MyDrive/ML/Datasets/all_fr_verse.npz


# Finetuning sur all couplets rap FR

**1ere phase 15k epochs**

In [ ]:
# entrainement en plusieurs fois sinon colab pas content

gpt2.finetune(sess,
              dataset = lyrics_npz, #lyrics_txt,             
              model_name = MODEL_in,
              checkpoint_dir = checkpoint_folder,
              run_name = MODEL_out,
              steps = 15000,
              #batch_size = 128,
              #learning_rate = 0.001,
              restore_from = 'fresh',
              print_every = 10,
              sample_every = 1000,
              sample_length = 300,
              save_every = 5000,
              #overwrite = True,
              reuse=True) #tf.AUTO_REUSE#


Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt
Loading dataset...


100%|██████████| 1/1 [00:02<00:00,  2.04s/it]


dataset has 64940854 tokens
Training...
[10 | 16.13] loss=3.78 avg=3.78
[20 | 28.83] loss=3.57 avg=3.67
[30 | 41.53] loss=3.46 avg=3.60
[40 | 54.23] loss=3.89 avg=3.67
[50 | 66.92] loss=3.60 avg=3.66
[60 | 79.62] loss=3.69 avg=3.66
[70 | 92.32] loss=3.56 avg=3.65
[80 | 105.01] loss=3.13 avg=3.58
[90 | 117.71] loss=3.43 avg=3.56
[100 | 130.41] loss=3.21 avg=3.53
[110 | 143.10] loss=3.49 avg=3.52
[120 | 155.80] loss=3.58 avg=3.53
[130 | 168.49] loss=3.42 avg=3.52
[140 | 181.18] loss=3.08 avg=3.49
[150 | 193.88] loss=3.44 avg=3.48
[160 | 206.57] loss=2.95 avg=3.45
[170 | 219.26] loss=3.29 avg=3.44
[180 | 231.96] loss=3.06 avg=3.41
[190 | 244.65] loss=3.31 avg=3.41
[200 | 257.35] loss=3.20 avg=3.40
[210 | 270.04] loss=3.00 avg=3.37
[220 | 282.74] loss=3.45 avg=3.38
[230 | 295.43] loss=3.20 avg=3.37
[240 | 308.13] loss=2.98 avg=3.35
[250 | 320.83] loss=3.30 avg=3.35
[260 | 333.52] loss=2.83 avg=3.33
[270 | 346.24] loss=3.15 avg=3.32
[280 | 358.94] loss=3.40 avg=3.32
[290 | 371.64] loss=3.41

**2e phase 15k -> 25 epochs**

In [ ]:
gpt2.finetune(sess,
              dataset = lyrics_npz, #lyrics_txt,             
              model_name = MODEL_in,
              checkpoint_dir = checkpoint_folder,
              run_name = MODEL_out,
              steps = 10000,
              #batch_size = 128,
              restore_from = 'latest',
              print_every = 10,
              sample_every = 1000,
              sample_length = 300,
              save_every = 5000,
              overwrite = True
              #reuse=True)


**3e phase 25k -> 35k epochs**

In [ ]:
# pas forcément indispensable : loss lente/plateau, voir les output intermédiaires (a priori pas de repet)

gpt2.finetune(sess,
              dataset = lyrics_npz, #lyrics_txt,             
              model_name = MODEL_in,
              checkpoint_dir = checkpoint_folder,
              run_name = MODEL_out,
              steps = 10000,
              #batch_size = 128,
              restore_from = 'latest',
              print_every = 10,
              sample_every = 1000,
              sample_length = 300,
              save_every = 5000,
              overwrite = True
              #reuse=True)

# Génération de couplets

In [ ]:
#graine = ""

gpt2.generate(sess,              
              checkpoint_dir = checkpoint_folder,
              run_name = MODEL_out,
              length = 200,
              temperature = 1,
              top_p = 0.85,
              #top_k = 40,
              prefix = "<|startoftext|>", #\n" + graine,
              truncate = "<|endoftext|>",
              #include_prefix = False,
              nsamples = 5,
              batch_size = 5)


<|startoftext|>
etre aussi fort qu'hier
tenter d'vivre dans cet hiver
etre aussi fort qu'hier
écraser par le passé
seul dans cette nuit éternelle
je peux perdre mon temps dans ce monde mais
faudrait qu'je change
j'ai dû faire partie de ce qu'ils m'ont donné
j'avais prévu des bons moments
mais des moments calmes

<|startoftext|>
ouais, ouais, c'est c'que les haineux me disent
qu'on s'applique toutes ces phrases qui sortent du tri
c'est ça, les mecs comme moi, je les connais par coeur
ils ont voulu m'expliquer le nom mais ça a pas leur place dans le rap
j'vais pas te l'dire, j'suis pas sûr d'le traiter
mais j'connais toutes les femmes comme ça, j'ai pas le temps d'faire le cliché
ouais, ouais, c'est ça, la fève est dépassée
pourquoi tu veux parler sur moi ?
même si tu fais pas le malin, j'te sou
<|startoftext|>
pas besoin de faire des conneries, j'fais des folies
j'ai fait des vidanges de ouf', sont aussi folles
pussy money sur mes loves, j'ai plus de followers
j'aime bien les québécois,

In [ ]:
#graine = ""

gpt2.generate(sess,              
              checkpoint_dir = checkpoint_folder,
              run_name = MODEL_out,
              length = 400,
              temperature = 0.85,
              top_p = 0.95,
              top_k = 40,
              prefix = "<|startoftext|>", #\n" + graine,
              truncate = "<|endoftext|>",
              #include_prefix = False,
              nsamples = 5,
              batch_size = 5)


<|startoftext|>
je suis pas un rappeur, j’ai une attitude à effet
je viens du 9.3.0.16, c’est de ma vie qu’on a fait
je suis le roi du son, mais ce serait pas pour autant
je laisse des fleurs sur mon macadam
mais la petite princesse a perdu ses habits
mais la petite princesse ne l’est pas quand elle est partie
la petite princesse ne l’est pas quand elle est partie
je ne sais pas ce que c’est que des photos
mais je ne sais pas ce que c’est que des photos
mais je ne sais pas ce que c’est que des photos
mais je ne sais pas ce que c’est que des photos
je ne sais pas ce que c’est que des photos
mais je ne sais pas ce que c’est que des photos
mais je ne sais pas ce que c’est que des photos
mais je ne sais pas ce que c’est que des photos
mais je ne sais pas ce que c’est que des photos
mais je ne sais pas ce que c’est que des photos
mais je ne sais pas ce que c’est que des photos
mais je ne sais pas ce que c’est que des photos
mais je ne sais pas ce que c’est que des photos
mais je ne sais pas

**Idem avec export en txt**

In [ ]:
#graine = ""

output_file = os.path.join(output_rep, MODEL_out + "_" + datetime.now().strftime("%m%d%Y %H:%M:%S") + ".txt")

gpt2.generate_to_file(sess,
                      destination_path = output_file,
                      checkpoint_dir= checkpoint_folder,
                      run_name = MODEL_out,
                      length = 200,
                      temperature = 1,
                      top_p = 0.95,
                      top_k = 40,
                      prefix = "<|startoftext|>", #\n" + graine,
                      truncate = "<|endoftext|>",
                      #include_prefix = False,
                      nsamples = 5,
                      batch_size = 5)